In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import itertools
import math
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Estimate & Actual Distance

##  Read Data

In [2]:
data = pd.read_csv('C:\\Users\\FESUB\\Python Study\\Inter_Data_New.csv', encoding='utf-8')
data = data.set_index('DATE')
data

,Real_Effective,SP500,BUSLOANS,Month_3,CPIAUCSL,CCI,Inflation,Housing,long_term,NFCI,M3,Unemployment_Rate,SMB,HML,RMW,CMA,Mkt_minus_RF,F_D,E_D,C_D
DATE,,,,,,,,,,,,,,,,,,,,
1975-07-01,113.86,88.750000,183.6510,6.55,54.000,98.68261,0.0972,11.72,8.0600,-6.475000e-01,9.750000e+11,8.60,3.42,1.70,0.43,1.19,-6.590000,23.289689,22.824515,7.541799
1975-08-01,115.23,86.879997,182.9282,6.87,54.200,98.69677,0.0860,10.95,8.4000,-5.660000e-01,9.830000e+11,8.40,-2.79,-0.90,1.03,-0.89,-2.850000,21.313321,19.279411,3.548020
1975-09-01,115.26,83.870003,182.8220,7.03,54.600,98.72337,0.0791,10.91,8.4300,-4.000000e-01,9.920000e+11,8.40,0.02,0.34,0.53,0.50,-4.260000,12.964091,10.751534,1.473532
1975-10-01,114.90,89.040001,182.7142,6.56,54.900,98.83214,0.0744,11.32,8.1400,-2.640000e-01,9.980000e+11,8.40,-4.23,0.30,-0.46,2.27,5.310000,17.675483,12.920998,1.819241
1975-11-01,115.19,91.239998,182.6189,6.26,55.300,99.07213,0.0738,11.41,8.0500,-2.825000e-01,1.010000e+12,8.30,-1.07,1.99,-0.62,1.71,2.640000,12.441139,6.276101,0.280815
1975-12-01,115.95,90.190002,183.3586,6.03,55.600,99.35911,0.0694,11.14,8.0000,-4.625000e-01,1.020000e+12,8.20,-0.03,1.76,-0.12,0.60,-1.600000,3.261455,2.921497,0.225094
1976-01-01,114.55,100.860001,182.6481,5.14,55.800,99.58169,0.0672,11.82,7.7400,-7.380000e-01,1.030000e+12,7.90,6.35,8.58,-1.90,2.36,12.160000,24.921450,16.160660,5.956648
1976-02-01,113.52,99.709999,182.0444,5.18,55.900,99.66043,0.0629,11.94,7.7900,-9.325000e-01,1.040000e+12,7.70,7.97,5.78,-2.58,3.84,0.320000,9.860273,4.636301,0.587561
1976-03-01,113.65,102.769997,179.4877,5.34,56.000,99.67143,0.0607,11.67,7.7300,-9.800000e-01,1.050000e+12,7.60,-1.34,-0.04,-0.38,1.09,2.320000,11.798600,7.398579,0.523751


## Normalization

In [3]:
def z_score_normalize(lst):
    normalized = []
    for value in lst:
        normalized_num = (value - np.mean(lst)) / np.std(lst)
        normalized.append(normalized_num)
    return normalized

In [4]:
data_columns = data.columns
x_data_columns = data.columns[[0,1,2,3,4,5,6,8,9,10,11,12,13,14,15,16]]
y_data_columns = data.columns[-3:]
y_data_columns

Index(['F_D', 'E_D', 'C_D'], dtype='object')

In [5]:
# 로그 취해줄 변수 
log_data_columns = data.columns[[0,1,2,10,11]]
log_data_columns

Index(['Real_Effective', 'SP500', 'BUSLOANS', 'M3', 'Unemployment_Rate'], dtype='object')

In [6]:
x_data = data[x_data_columns]
x_data

,Real_Effective,SP500,BUSLOANS,Month_3,CPIAUCSL,CCI,Inflation,long_term,NFCI,M3,Unemployment_Rate,SMB,HML,RMW,CMA,Mkt_minus_RF
DATE,,,,,,,,,,,,,,,,
1975-07-01,113.86,88.750000,183.6510,6.55,54.000,98.68261,0.0972,8.0600,-6.475000e-01,9.750000e+11,8.60,3.42,1.70,0.43,1.19,-6.590000
1975-08-01,115.23,86.879997,182.9282,6.87,54.200,98.69677,0.0860,8.4000,-5.660000e-01,9.830000e+11,8.40,-2.79,-0.90,1.03,-0.89,-2.850000
1975-09-01,115.26,83.870003,182.8220,7.03,54.600,98.72337,0.0791,8.4300,-4.000000e-01,9.920000e+11,8.40,0.02,0.34,0.53,0.50,-4.260000
1975-10-01,114.90,89.040001,182.7142,6.56,54.900,98.83214,0.0744,8.1400,-2.640000e-01,9.980000e+11,8.40,-4.23,0.30,-0.46,2.27,5.310000
1975-11-01,115.19,91.239998,182.6189,6.26,55.300,99.07213,0.0738,8.0500,-2.825000e-01,1.010000e+12,8.30,-1.07,1.99,-0.62,1.71,2.640000
1975-12-01,115.95,90.190002,183.3586,6.03,55.600,99.35911,0.0694,8.0000,-4.625000e-01,1.020000e+12,8.20,-0.03,1.76,-0.12,0.60,-1.600000
1976-01-01,114.55,100.860001,182.6481,5.14,55.800,99.58169,0.0672,7.7400,-7.380000e-01,1.030000e+12,7.90,6.35,8.58,-1.90,2.36,12.160000
1976-02-01,113.52,99.709999,182.0444,5.18,55.900,99.66043,0.0629,7.7900,-9.325000e-01,1.040000e+12,7.70,7.97,5.78,-2.58,3.84,0.320000
1976-03-01,113.65,102.769997,179.4877,5.34,56.000,99.67143,0.0607,7.7300,-9.800000e-01,1.050000e+12,7.60,-1.34,-0.04,-0.38,1.09,2.320000


In [7]:
y_data = data[y_data_columns]
y_data

,F_D,E_D,C_D
DATE,,,
1975-07-01,23.289689,22.824515,7.541799
1975-08-01,21.313321,19.279411,3.548020
1975-09-01,12.964091,10.751534,1.473532
1975-10-01,17.675483,12.920998,1.819241
1975-11-01,12.441139,6.276101,0.280815
1975-12-01,3.261455,2.921497,0.225094
1976-01-01,24.921450,16.160660,5.956648
1976-02-01,9.860273,4.636301,0.587561
1976-03-01,11.798600,7.398579,0.523751


In [8]:
### ln 취하기 
# np.log 는 자연로그를 반환함
for column in log_data_columns:
    x_data[column] = np.log(x_data[column])

x_data

C:\Users\FESUB\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,Real_Effective,SP500,BUSLOANS,Month_3,CPIAUCSL,CCI,Inflation,long_term,NFCI,M3,Unemployment_Rate,SMB,HML,RMW,CMA,Mkt_minus_RF
DATE,,,,,,,,,,,,,,,,
1975-07-01,4.734970,4.485823,5.213037,6.55,54.000,98.68261,0.0972,8.0600,-6.475000e-01,27.605703,2.151762,3.42,1.70,0.43,1.19,-6.590000
1975-08-01,4.746930,4.464528,5.209094,6.87,54.200,98.69677,0.0860,8.4000,-5.660000e-01,27.613875,2.128232,-2.79,-0.90,1.03,-0.89,-2.850000
1975-09-01,4.747190,4.429268,5.208513,7.03,54.600,98.72337,0.0791,8.4300,-4.000000e-01,27.622989,2.128232,0.02,0.34,0.53,0.50,-4.260000
1975-10-01,4.744062,4.489086,5.207923,6.56,54.900,98.83214,0.0744,8.1400,-2.640000e-01,27.629019,2.128232,-4.23,0.30,-0.46,2.27,5.310000
1975-11-01,4.746583,4.513493,5.207401,6.26,55.300,99.07213,0.0738,8.0500,-2.825000e-01,27.640971,2.116256,-1.07,1.99,-0.62,1.71,2.640000
1975-12-01,4.753159,4.501919,5.211444,6.03,55.600,99.35911,0.0694,8.0000,-4.625000e-01,27.650824,2.104134,-0.03,1.76,-0.12,0.60,-1.600000
1976-01-01,4.741011,4.613733,5.207561,5.14,55.800,99.58169,0.0672,7.7400,-7.380000e-01,27.660580,2.066863,6.35,8.58,-1.90,2.36,12.160000
1976-02-01,4.731979,4.602266,5.204251,5.18,55.900,99.66043,0.0629,7.7900,-9.325000e-01,27.670242,2.041220,7.97,5.78,-2.58,3.84,0.320000
1976-03-01,4.733124,4.632493,5.190107,5.34,56.000,99.67143,0.0607,7.7300,-9.800000e-01,27.679811,2.028148,-1.34,-0.04,-0.38,1.09,2.320000


In [9]:
### log 이후 정규화
#for column in x_data_columns:
#    x_data[column] = z_score_normalize(x_data[column])
#
#x_data

## Feature Selection

###  Forward Selection

In [10]:
def forward(X, y, predictors):
    # 데이터 변수들이 미리정의된 predictors에 있는지 없는지 확인 및 분류
    # 남아있는 변수들 중에 어떤 변수들을 쓰면 가장 낮은 AIC를 도출할 수 있는지 찾는 코드
    remaining_predictors = [p for p in X.columns if p not in predictors]
    
    results = [] #results 빈 리스트 선언
    for p in remaining_predictors:
        #x와 y그리고 p 번재 predictor, 상수항 추가해서 regression 돌리기 & results에 append하기
        results.append(processSubset(X=X, y= y, feature_set=predictors+[p]+[]))
    # 데이터프레임으로 변환
    models = pd.DataFrame(results)

    # AIC가 가장 낮은 것을 선택
    best_model = models.loc[models['AIC'].argmin()] # index
    print(models['AIC'])
    print("Processed ", models.shape[0], "models on", len(predictors)+1)
    print('Selected predictors:',best_model['model'].model.exog_names,' AIC:',best_model[1] )
    return best_model


#### 전진선택법 모델

def forward_model(X,y):
    Fmodels = pd.DataFrame(columns=["AIC", "model"])

    # 미리 정의된 데이터 변수
    predictors = []
    # 변수 1~10개 : 0~9 -> 1~10
    for i in range(1, len(X.columns) + 1):
        Forward_result = forward(X=X,y=y,predictors=predictors)
        if i > 1:
            if Forward_result['AIC'] > Fmodel_before:
                break
        Fmodels.loc[i] = Forward_result
        predictors = Fmodels.loc[i]["model"].model.exog_names
        Fmodel_before = Fmodels.loc[i]["AIC"]
        predictors = [ k for k in predictors if k != 'const']

    return(Fmodels['model'][len(Fmodels['model'])])

### Backward Elimination

In [11]:
def processSubset(X,y, feature_set):
            model = sm.OLS(y,sm.add_constant(X[list(feature_set)])) # Modeling
            regr = model.fit() # 모델 학습
            AIC = regr.aic # 모델의 AIC
            return {"model":regr, "AIC":AIC}
        

processSubset(X=x_data, y=y_data['E_D'], feature_set = x_data.columns)

C:\Users\FESUB\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


{'model': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x1ed7d39d390>,
 'AIC': 3876.8310399664933}

In [12]:
######## 후진선택법(step=1)
def backward(X,y,predictors):
    results = []
    # 데이터 변수들이 미리정의된 predictors 조합 확인
    for combo in itertools.combinations(predictors, len(predictors) - 1):
        results.append(processSubset(X=X, y= y,feature_set=combo))
    models = pd.DataFrame(results)
                       
    # 가장 낮은 AIC를 가진 모델을 선택
    best_model = models.loc[models['AIC'].argmin()]
    print(models['AIC'])
    print("Processed ", models.shape[0], "models on", len(predictors) - 1)
    print('Selected predictors:',best_model['model'].model.exog_names,' AIC:',best_model[1] )
    return best_model





# 후진 소거법 모델
def backward_model(X, y):
    Bmodels = pd.DataFrame(columns=["AIC", "model"], index = range(1,len(X.columns)))
    predictors = X.columns
    Bmodel_before = processSubset(X,y,predictors)['AIC']
    while (len(predictors) > 1):
        Backward_result = backward(X=X, y= y, predictors = predictors)
        if Backward_result['AIC'] > Bmodel_before:
            break
        Bmodels.loc[len(predictors) - 1] = Backward_result
        predictors = Bmodels.loc[len(predictors) - 1]["model"].model.exog_names
        Bmodel_before = Backward_result['AIC']
        predictors = [ k for k in predictors if k != 'const']
    return (Bmodels['model'].dropna().iloc[0])

### Stepwise Selection

In [13]:
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.argmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

## Regression

### Euclidean Distance

In [14]:
ED_data = pd.concat([x_data, y_data['E_D']],axis = 1)
ED_data

,Real_Effective,SP500,BUSLOANS,Month_3,CPIAUCSL,CCI,Inflation,long_term,NFCI,M3,Unemployment_Rate,SMB,HML,RMW,CMA,Mkt_minus_RF,E_D
DATE,,,,,,,,,,,,,,,,,
1975-07-01,4.734970,4.485823,5.213037,6.55,54.000,98.68261,0.0972,8.0600,-6.475000e-01,27.605703,2.151762,3.42,1.70,0.43,1.19,-6.590000,22.824515
1975-08-01,4.746930,4.464528,5.209094,6.87,54.200,98.69677,0.0860,8.4000,-5.660000e-01,27.613875,2.128232,-2.79,-0.90,1.03,-0.89,-2.850000,19.279411
1975-09-01,4.747190,4.429268,5.208513,7.03,54.600,98.72337,0.0791,8.4300,-4.000000e-01,27.622989,2.128232,0.02,0.34,0.53,0.50,-4.260000,10.751534
1975-10-01,4.744062,4.489086,5.207923,6.56,54.900,98.83214,0.0744,8.1400,-2.640000e-01,27.629019,2.128232,-4.23,0.30,-0.46,2.27,5.310000,12.920998
1975-11-01,4.746583,4.513493,5.207401,6.26,55.300,99.07213,0.0738,8.0500,-2.825000e-01,27.640971,2.116256,-1.07,1.99,-0.62,1.71,2.640000,6.276101
1975-12-01,4.753159,4.501919,5.211444,6.03,55.600,99.35911,0.0694,8.0000,-4.625000e-01,27.650824,2.104134,-0.03,1.76,-0.12,0.60,-1.600000,2.921497
1976-01-01,4.741011,4.613733,5.207561,5.14,55.800,99.58169,0.0672,7.7400,-7.380000e-01,27.660580,2.066863,6.35,8.58,-1.90,2.36,12.160000,16.160660
1976-02-01,4.731979,4.602266,5.204251,5.18,55.900,99.66043,0.0629,7.7900,-9.325000e-01,27.670242,2.041220,7.97,5.78,-2.58,3.84,0.320000,4.636301
1976-03-01,4.733124,4.632493,5.190107,5.34,56.000,99.67143,0.0607,7.7300,-9.800000e-01,27.679811,2.028148,-1.34,-0.04,-0.38,1.09,2.320000,7.398579


#### Euclidean_All

In [15]:
# 모든 변수 적용했을 때
ED_model_all = smf.ols(formula = 'E_D ~  Real_Effective + SP500 + BUSLOANS + Month_3 + CPIAUCSL + CCI + Inflation + long_term + NFCI + M3 + Unemployment_Rate + SMB + HML + RMW + CMA + Mkt_minus_RF',data = ED_data)
ED_result_all = ED_model_all.fit()
ED_result_all.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    E_D   R-squared:                       0.314
Model:                            OLS   Adj. R-squared:                  0.294
Method:                 Least Squares   F-statistic:                     15.83
Date:                Sat, 05 Dec 2020   Prob (F-statistic):           5.98e-34
Time:                        20:36:50   Log-Likelihood:                -1922.4
No. Observations:                 534   AIC:                             3877.
Df Residuals:                     518   BIC:                             3945.
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept           652.1668    222.926      2.925      0.004     214.217    1090.117
Real_Effective        2.8357      4.955      0.572      0.567      -6.899      12.570
SP500                -0.3753      3.400     -0.110      0.912      -7.055       6.304
BUSLOANS             40.6174      6.293      6.454      0.000      28.254      52.981
Month_3               0.1750      0.444      0.394      0.694      -0.698       1.048
CPIAUCSL             -0.1652      0.119     -1.393      0.164      -0.398       0.068
CCI                  -1.4732      0.719     -2.048      0.041      -2.887      -0.060
Inflation            51.0433     39.970      1.277      0.202     -27.480     129.567
long_term            -2.1962      0.587     -3.740      0.000      -3.350      -1.042
NFCI              -5.733e-11   1.62e-11     -3.535      0.000   -8.92e-11   -2.55e-11
M3                  -25.9046      9.132     -2.837      0.005     -43.846      -7.964
Unemployment_Rate     9.7438      3.937      2.475      0.014       2.010      17.477
SMB                  -0.4523      0.153     -2.953      0.003      -0.753      -0.151
HML                   0.3671      0.195      1.883      0.060      -0.016       0.750
RMW                  -0.4639      0.200     -2.323      0.021      -0.856      -0.072
CMA                  -0.7929      0.298     -2.658      0.008      -1.379      -0.207
Mkt_minus_RF         -1.0572      0.103    -10.241      0.000      -1.260      -0.854
==============================================================================
Omnibus:                      466.661   Durbin-Watson:                   1.703
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            18771.708
Skew:                           3.589   Prob(JB):                         0.00
Kurtosis:                      31.145   Cond. No.                     3.80e+14
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.8e+14. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

#### Euclidean_Forward

In [16]:
A = forward(X=x_data, y=y_data['E_D'],predictors=[])

0     4048.421203
1     4046.857180
2     4041.749185
3     4048.573076
4     4044.528965
5     4049.020895
6     4049.538924
7     4045.363088
8     4050.285069
9     4048.706838
10    4050.212994
11    4033.036605
12    4048.279462
13    4044.369779
14    4046.319774
15    3956.106364
Name: AIC, dtype: float64
Processed  16 models on 1
Selected predictors: ['const', 'Mkt_minus_RF']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000001ED7EC9FEF0>


C:\Users\FESUB\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: 
The current behaviour of 'Series.argmin' is deprecated, use 'idxmin'
instead.
The behavior of 'argmin' will be corrected to return the positional
minimum in the future. For now, use 'series.values.argmin' or
'np.argmin(np.array(values))' to get the position of the minimum
row.
  


In [17]:
print(A)

AIC                                                3956.11
model    <statsmodels.regression.linear_model.Regressio...
Name: 15, dtype: object


In [18]:
ED_model_forward = forward_model(X=x_data, y=y_data['E_D'])

0     4048.421203
1     4046.857180
2     4041.749185
3     4048.573076
4     4044.528965
5     4049.020895
6     4049.538924
7     4045.363088
8     4050.285069
9     4048.706838
10    4050.212994
11    4033.036605
12    4048.279462
13    4044.369779
14    4046.319774
15    3956.106364
Name: AIC, dtype: float64
Processed  16 models on 1
Selected predictors: ['const', 'Mkt_minus_RF']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000001ED7ECB09E8>
0     3955.120364
1     3952.026493
2     3946.398396
3     3953.044023
4     3949.623242
5     3952.385090
6     3955.148732
7     3948.545011
8     3956.498253
9     3957.966286
10    3957.955823
11    3953.118639
12    3957.116484
13    3958.044204
14    3955.637701
Name: AIC, dtype: float64

C:\Users\FESUB\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: 
The current behaviour of 'Series.argmin' is deprecated, use 'idxmin'
instead.
The behavior of 'argmin' will be corrected to return the positional
minimum in the future. For now, use 'series.values.argmin' or
'np.argmin(np.array(values))' to get the position of the minimum
row.
  



Processed  15 models on 2
Selected predictors: ['const', 'Mkt_minus_RF', 'BUSLOANS']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000001ED7EC9F1D0>
0     3945.849787
1     3938.187239
2     3948.320683
3     3943.593931
4     3942.607121
5     3947.322684
6     3948.093432
7     3947.312560
8     3946.629129
9     3943.713770
10    3944.603116
11    3947.913600
12    3948.265747
13    3946.348146
Name: AIC, dtype: float64
Processed  14 models on 3
Selected predictors: ['const', 'Mkt_minus_RF', 'BUSLOANS', 'SP500']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000001ED7EC90B70>
0     3933.975078
1     3938.845921
2     3939.199617
3     3936.050253
4     3939.829276
5     3931.687150
6     3939.146148
7     3938.635939
8     3939.223359
9     3936.981159
10    3939.929928
11    3939.929313
12    3938.317654
Name: AIC, dtype: float64
Processed  13 models on 4
Selected predictors: ['const', 'Mkt_minus_RF', 'BUSLOANS', '

In [19]:
ED_model_forward.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    E_D   R-squared:                       0.314
Model:                            OLS   Adj. R-squared:                  0.296
Method:                 Least Squares   F-statistic:                     16.97
Date:                Sat, 05 Dec 2020   Prob (F-statistic):           1.53e-34
Time:                        20:36:51   Log-Likelihood:                -1922.5
No. Observations:                 534   AIC:                             3875.
Df Residuals:                     519   BIC:                             3939.
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const               679.8183    211.417      3.216      0.001     264.480    1095.156
Mkt_minus_RF         -1.0553      0.103    -10.242      0.000      -1.258      -0.853
BUSLOANS             41.3915      5.974      6.929      0.000      29.656      53.127
SP500                -0.6277      3.336     -0.188      0.851      -7.182       5.927
long_term            -2.0269      0.400     -5.068      0.000      -2.812      -1.241
CPIAUCSL             -0.1554      0.116     -1.341      0.180      -0.383       0.072
Real_Effective        2.7375      4.945      0.554      0.580      -6.977      12.452
Unemployment_Rate     9.2223      3.705      2.489      0.013       1.945      16.500
Inflation            57.5211     36.401      1.580      0.115     -13.991     129.033
CCI                  -1.4676      0.719     -2.042      0.042      -2.880      -0.056
NFCI              -5.943e-11   1.53e-11     -3.881      0.000   -8.95e-11   -2.93e-11
M3                  -27.0158      8.679     -3.113      0.002     -44.066      -9.966
SMB                  -0.4610      0.151     -3.043      0.002      -0.759      -0.163
RMW                  -0.4746      0.198     -2.401      0.017      -0.863      -0.086
CMA                  -0.7869      0.298     -2.644      0.008      -1.372      -0.202
HML                   0.3644      0.195      1.871      0.062      -0.018       0.747
==============================================================================
Omnibus:                      466.109   Durbin-Watson:                   1.703
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            18661.611
Skew:                           3.584   Prob(JB):                         0.00
Kurtosis:                      31.060   Cond. No.                     3.60e+14
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.6e+14. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [20]:
# Forward Selection으로 변수 선택했을 때
ED_model_forward = smf.ols(formula = 'E_D ~ Mkt_minus_RF+BUSLOANS+CPIAUCSL+SP500+long_term+Unemployment_Rate+Real_Effective+Inflation+CCI+NFCI+M3+SMB+RMW+CMA+HML',data = ED_data)
ED_result_forward = ED_model_forward.fit()
ED_result_forward.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    E_D   R-squared:                       0.314
Model:                            OLS   Adj. R-squared:                  0.296
Method:                 Least Squares   F-statistic:                     16.97
Date:                Sat, 05 Dec 2020   Prob (F-statistic):           1.53e-34
Time:                        20:36:51   Log-Likelihood:                -1922.5
No. Observations:                 534   AIC:                             3875.
Df Residuals:                     519   BIC:                             3939.
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept           679.8185    211.417      3.216      0.001     264.481    1095.156
Mkt_minus_RF         -1.0553      0.103    -10.242      0.000      -1.258      -0.853
BUSLOANS             41.3915      5.974      6.929      0.000      29.656      53.127
CPIAUCSL             -0.1554      0.116     -1.341      0.180      -0.383       0.072
SP500                -0.6278      3.336     -0.188      0.851      -7.182       5.927
long_term            -2.0269      0.400     -5.068      0.000      -2.812      -1.241
Unemployment_Rate     9.2223      3.705      2.489      0.013       1.945      16.500
Real_Effective        2.7375      4.945      0.554      0.580      -6.977      12.452
Inflation            57.5211     36.401      1.580      0.115     -13.991     129.033
CCI                  -1.4676      0.719     -2.042      0.042      -2.880      -0.056
NFCI              -5.943e-11   1.53e-11     -3.881      0.000   -8.95e-11   -2.93e-11
M3                  -27.0158      8.679     -3.113      0.002     -44.066      -9.966
SMB                  -0.4610      0.151     -3.043      0.002      -0.759      -0.163
RMW                  -0.4746      0.198     -2.401      0.017      -0.863      -0.086
CMA                  -0.7869      0.298     -2.644      0.008      -1.372      -0.202
HML                   0.3644      0.195      1.871      0.062      -0.018       0.747
==============================================================================
Omnibus:                      466.109   Durbin-Watson:                   1.703
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            18661.614
Skew:                           3.584   Prob(JB):                         0.00
Kurtosis:                      31.060   Cond. No.                     3.60e+14
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.6e+14. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

#### Euclidean_Backward

In [21]:
ED_model_backward = backward_model(X=x_data, y=y_data['E_D'])

0     3973.279684
1     3882.067349
2     3880.365754
3     3878.471947
4     3883.744778
5     3881.109711
6     3885.062038
7     3889.557043
8     3889.056620
9     3876.509611
10    3879.136334
11    3878.828644
12    3874.991014
13    3916.135513
14    3874.843605
15    3875.168562
Name: AIC, dtype: float64
Processed  16 models on 15
Selected predictors: ['const', 'Real_Effective', 'BUSLOANS', 'Month_3', 'CPIAUCSL', 'CCI', 'Inflation', 'long_term', 'NFCI', 'M3', 'Unemployment_Rate', 'SMB', 'HML', 'RMW', 'CMA', 'Mkt_minus_RF']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000001ED7EC90320>
0     3973.678416
1     3880.159108
2     3878.378937
3     3876.491105
4     3881.803195
5     3879.833875
6     3887.246459
7     3891.399414
8     3887.235003
9     3874.511828
10    3879.067205
11    3879.785763
12    3873.027424
13    3923.446618
14    3873.263139
Name: AIC, dtype: float64
Processed  15 models on 14
Selected predictors: ['const', 'Real_Effec

C:\Users\FESUB\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: 
The current behaviour of 'Series.argmin' is deprecated, use 'idxmin'
instead.
The behavior of 'argmin' will be corrected to return the positional
minimum in the future. For now, use 'series.values.argmin' or
'np.argmin(np.array(values))' to get the position of the minimum
row.
  # Remove the CWD from sys.path while we load stuff.


In [22]:
ED_model_backward.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    E_D   R-squared:                       0.314
Model:                            OLS   Adj. R-squared:                  0.297
Method:                 Least Squares   F-statistic:                     18.31
Date:                Sat, 05 Dec 2020   Prob (F-statistic):           3.54e-35
Time:                        20:36:51   Log-Likelihood:                -1922.5
No. Observations:                 534   AIC:                             3873.
Df Residuals:                     520   BIC:                             3933.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const               658.8347    179.421      3.672      0.000     306.356    1011.313
Real_Effective        3.0230      4.702      0.643      0.521      -6.214      12.260
BUSLOANS             40.9193      5.415      7.556      0.000      30.281      51.558
CPIAUCSL             -0.1713      0.079     -2.165      0.031      -0.327      -0.016
CCI                  -1.5388      0.611     -2.519      0.012      -2.739      -0.339
Inflation            57.1762     36.322      1.574      0.116     -14.179     128.531
long_term            -1.9955      0.363     -5.495      0.000      -2.709      -1.282
NFCI              -5.804e-11   1.34e-11     -4.328      0.000   -8.44e-11   -3.17e-11
M3                  -26.0545      7.008     -3.718      0.000     -39.822     -12.287
Unemployment_Rate     9.3874      3.596      2.611      0.009       2.323      16.452
SMB                  -0.4630      0.151     -3.067      0.002      -0.760      -0.166
HML                   0.3646      0.195      1.874      0.061      -0.018       0.747
RMW                  -0.4756      0.197     -2.410      0.016      -0.863      -0.088
CMA                  -0.7904      0.297     -2.664      0.008      -1.373      -0.207
Mkt_minus_RF         -1.0581      0.102    -10.388      0.000      -1.258      -0.858
==============================================================================
Omnibus:                      466.037   Durbin-Watson:                   1.703
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            18614.919
Skew:                           3.584   Prob(JB):                         0.00
Kurtosis:                      31.022   Cond. No.                     3.06e+14
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.06e+14. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [23]:
ED_model_backward = smf.ols(formula = 'E_D ~ BUSLOANS+CPIAUCSL+CCI+Inflation+long_term+NFCI+M3+Unemployment_Rate+SMB+HML+RMW+CMA+Mkt_minus_RF', data = ED_data)
ED_result_backward = ED_model_backward.fit()
ED_result_backward.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    E_D   R-squared:                       0.313
Model:                            OLS   Adj. R-squared:                  0.296
Method:                 Least Squares   F-statistic:                     18.26
Date:                Sat, 05 Dec 2020   Prob (F-statistic):           4.31e-35
Time:                        20:36:52   Log-Likelihood:                -1922.7
No. Observations:                 534   AIC:                             3873.
Df Residuals:                     520   BIC:                             3933.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept           637.9308    176.515      3.614      0.000     291.161     984.701
BUSLOANS             41.1518      5.405      7.613      0.000      30.533      51.771
CPIAUCSL             -0.1812      0.078     -2.334      0.020      -0.334      -0.029
CCI                  -1.4136      0.579     -2.441      0.015      -2.551      -0.276
Inflation            59.2344     36.191      1.637      0.102     -11.865     130.334
long_term            -1.9581      0.359     -5.460      0.000      -2.663      -1.254
NFCI              -5.574e-11   1.29e-11     -4.312      0.000   -8.11e-11   -3.03e-11
M3                  -25.3095      6.915     -3.660      0.000     -38.893     -11.726
Unemployment_Rate     9.7504      3.553      2.744      0.006       2.771      16.730
SMB                  -0.4566      0.151     -3.030      0.003      -0.753      -0.161
HML                   0.3635      0.195      1.868      0.062      -0.019       0.746
RMW                  -0.4684      0.197     -2.376      0.018      -0.856      -0.081
CMA                  -0.7848      0.297     -2.645      0.008      -1.368      -0.202
Mkt_minus_RF         -1.0602      0.102    -10.410      0.000      -1.260      -0.860
==============================================================================
Omnibus:                      464.262   Durbin-Watson:                   1.702
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            18288.683
Skew:                           3.568   Prob(JB):                         0.00
Kurtosis:                      30.768   Cond. No.                     3.01e+14
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.01e+14. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

#### Euclidean_Stepwise

In [24]:
ED_result_stepwise = stepwise_selection(x_data, y_data['E_D'])

C:\Users\FESUB\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: 
The current behaviour of 'Series.argmin' is deprecated, use 'idxmin'
instead.
The behavior of 'argmin' will be corrected to return the positional
minimum in the future. For now, use 'series.values.argmin' or
'np.argmin(np.array(values))' to get the position of the minimum
row.


Add  Mkt_minus_RF                   with p-value 3.59501e-22
Add  BUSLOANS                       with p-value 0.000648528
Add  SP500                          with p-value 0.00146262
Add  long_term                      with p-value 0.00372638
Add  CPIAUCSL                       with p-value 0.00330344


In [25]:
ED_model_Stepwise= smf.ols(formula = 'E_D ~ SP500 + long_term  +  Mkt_minus_RF + CPIAUCSL + BUSLOANS', data = ED_data)
ED_result_Stepwise = ED_model_Stepwise.fit()
ED_result_Stepwise.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    E_D   R-squared:                       0.221
Model:                            OLS   Adj. R-squared:                  0.214
Method:                 Least Squares   F-statistic:                     29.95
Date:                Sat, 05 Dec 2020   Prob (F-statistic):           8.09e-27
Time:                        20:36:52   Log-Likelihood:                -1956.5
No. Observations:                 534   AIC:                             3925.
Df Residuals:                     528   BIC:                             3951.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept      -43.9384     14.987     -2.932      0.004     -73.380     -14.497
SP500           -7.3571      1.799     -4.088      0.000     -10.892      -3.822
long_term       -1.2530      0.310     -4.039      0.000      -1.862      -0.644
Mkt_minus_RF    -0.9757      0.094    -10.414      0.000      -1.160      -0.792
CPIAUCSL        -0.1328      0.045     -2.951      0.003      -0.221      -0.044
BUSLOANS        19.7918      3.820      5.181      0.000      12.287      27.297
==============================================================================
Omnibus:                      464.957   Durbin-Watson:                   1.571
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            16208.755
Skew:                           3.630   Prob(JB):                         0.00
Kurtosis:                      28.996   Cond. No.                     6.39e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.39e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

#### VIF Factor Analysis

In [26]:
vif_ED_forward = x_data[['Housing', 'NFCI', 'RF', 'SP500', 'BUSLOANS', 'long_term', 'Month_3', 'SMB']]
vif_ED_backward = x_data[['SP500', 'BUSLOANS', 'Housing', 'long_term', 'NFCI', 'RF']]
vif_ED_stepwise = x_data[['Housing', 'NFCI', 'RF','SP500','BUSLOANS', 'long_term' ]]
# Stepwise의 경우 forward와 결과 동일 
# vif_ED_Stepwise = x_data[['Housing', 'NFCI', 'RF', 'CPIAUCSL']]    

KeyError: "['Housing', 'RF'] not in index"

In [ ]:
## ED , Forward selection의 Vif 결과
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(
    vif_ED_forward.values, i) for i in range(vif_ED_forward.shape[1])]
vif["features"] = vif_ED_forward.columns
vif.round(2)

In [ ]:
## Month_3 변수 제거후(vif_ED_forward) VIF 확인 

vif = pd.DataFrame()
vif_ED_forward= vif_ED_forward.drop('Month_3',axis=1)
vif["VIF Factor"] = [variance_inflation_factor(
    vif_ED_forward.values, i) for i in range(vif_ED_forward.shape[1])]
vif["features"] = vif_ED_forward.columns
vif
## 모두 10 이하의 VIF 값을 갖는다

In [ ]:
## SP500 변수 제거후(vif_ED_forward) VIF 확인 

vif = pd.DataFrame()
vif_ED_forward= vif_ED_forward.drop('SP500',axis=1)
vif["VIF Factor"] = [variance_inflation_factor(
    vif_ED_forward.values, i) for i in range(vif_ED_forward.shape[1])]
vif["features"] = vif_ED_forward.columns
vif
## 모두 10 이하의 VIF 값을 갖는다

In [ ]:
## ED , Backward selection의 Vif 결과
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(
    vif_ED_backward.values, i) for i in range(vif_ED_backward.shape[1])]
vif["features"] = vif_ED_backward.columns
vif.round(2)

In [ ]:
## SP500 변수 제거후(vif_ED_backward) VIF 확인 

vif = pd.DataFrame()
vif_ED_backward= vif_ED_backward.drop('SP500',axis=1)
vif["VIF Factor"] = [variance_inflation_factor(
    vif_ED_backward.values, i) for i in range(vif_ED_backward.shape[1])]
vif["features"] = vif_ED_backward.columns
vif
## 모두 10 이하의 VIF 값을 갖는다

In [ ]:
## ED , Stepwise selection의 Vif 결과
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(
    vif_ED_stepwise.values, i) for i in range(vif_ED_stepwise.shape[1])]
vif["features"] = vif_ED_stepwise.columns
vif.round(2)

In [ ]:
## SP500 변수 제거후(vif_ED_backward) VIF 확인 

vif = pd.DataFrame()
vif_ED_stepwise= vif_ED_stepwise.drop('SP500',axis=1)
vif["VIF Factor"] = [variance_inflation_factor(
    vif_ED_stepwise.values, i) for i in range(vif_ED_stepwise.shape[1])]
vif["features"] = vif_ED_stepwise.columns
vif
## 모두 10 이하의 VIF 값을 갖는다

In [ ]:
################ 우선 ED만 진행하였습니다. 

In [ ]:
################ 우선 ED만 진행하였습니다. 

### Cosine Distance

In [ ]:
CD_data = pd.concat([x_data, y_data['C_D']],axis = 1)
CD_data

#### Cosine_All

In [ ]:
CD_model_all = smf.ols(formula = 'C_D ~ Real_Effective + SP500 + BUSLOANS + Month_3 + CPIAUCSL + CCI + Inflation + Housing + long_term + NFCI + M3 + Unemployment_Rate + SMB + HML + RMW + CMA + RF',data = CD_data)
CD_result_all = CD_model_all.fit()
CD_result_all.summary()

#### Cosine_Forward

In [ ]:
CD_model = forward_model(X=x_data, y=y_data['C_D'])

In [ ]:
CD_model_forward = smf.ols(formula = 'C_D ~ CPIAUCSL + HML',data = CD_data)
CD_result_forward = CD_model_forward.fit()
CD_result_forward.summary()

#### Cosine_Backward

In [ ]:
CD_model_backward = backward_model(X=x_data, y=y_data['C_D'])

In [ ]:
CD_model_backward = smf.ols(formula = 'C_D ~ CPIAUCSL + long_term + NFCI', data = CD_data)
CD_result_backward = CD_model_backward.fit()
CD_result_backward.summary()

### Cosine_Stepwise

In [ ]:
CD_result_stepwise = stepwise_selection(x_data, y_data['C_D'])

In [ ]:
CD_model_Stepwise= smf.ols(formula = 'C_D ~CPIAUCSL', data = CD_data)
CD_result_Stepwise = CD_model_Stepwise.fit()
CD_result_Stepwise.summary()

#### VIF Factor Analysis

In [ ]:
vif_CD_forward = x_data[['CPIAUCSL','HML']]
vif_CD_backward = x_data[['CPIAUCSL','long_term','NFCI']]
vif_CD_Stepwise = x_data[['CPIAUCSL']]

In [ ]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(
    vif_CD_forward.values, i) for i in range(vif_CD_forward.shape[1])]
vif["features"] = vif_CD_forward.columns
vif.round(2)

In [ ]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(
    vif_CD_backward.values, i) for i in range(vif_CD_backward.shape[1])]
vif["features"] = vif_CD_backward.columns
vif.round(2)

In [ ]:
#vif = pd.DataFrame()
#vif["VIF Factor"] = [variance_inflation_factor(
#    vif_CD_Stepwise.values, i) for i in range(vif_CD_Stepwise.shape[1])]
#vif["features"] = vif_CD_Stepwise.columns
#vif.round(2)

### Frechet Distance

In [ ]:
FD_data = pd.concat([x_data, y_data['F_D']], axis = 1)
FD_data

#### Frechet_All

In [ ]:
FD_model_all = smf.ols(formula = 'F_D ~  Real_Effective + SP500 + BUSLOANS + CPIAUCSL + CCI + Inflation + Housing + long_term + NFCI + M3 + Unemployment_Rate + SMB + HML + RMW + CMA + RF',data = FD_data)
FD_result_all = FD_model_all.fit()
FD_result_all.summary()

#### Frechet_Forward

In [ ]:
FD_model = forward_model(X=x_data, y=y_data['F_D'])

In [ ]:
FD_model_forward = smf.ols(formula = 'F_D ~  Housing + NFCI + RF + CPIAUCSL + HML + BUSLOANS + SP500 + M3',data = FD_data)
FD_result_forward = FD_model_forward.fit()
FD_result_forward.summary()

#### Frechet_Backward

In [ ]:
FD_model_backward = backward_model(X=x_data, y=y_data['F_D'])

In [ ]:
FD_model_backward = smf.ols(formula = 'F_D ~ SP500 + BUSLOANS + CPIAUCSL + Housing + NFCI + HML + RF', data = FD_data)
FD_result_backward = FD_model_backward.fit()
FD_result_backward.summary()

### Frechet_Stepwise

In [ ]:
FD_result_stepwise = stepwise_selection(x_data, y_data['F_D'])

In [ ]:
FD_model_Stepwise= smf.ols(formula = 'F_D ~ Housing + NFCI + RF + CPIAUCSL + HML', data = FD_data)
FD_result_Stepwise = FD_model_Stepwise.fit()
FD_result_Stepwise.summary()

#### VIF Factor Analysis

In [ ]:
vif_FD_forward = x_data[['Housing', 'NFCI', 'RF', 'CPIAUCSL', 'HML', 'BUSLOANS', 'SP500', 'M3']]
vif_FD_backward = x_data[ ['SP500', 'BUSLOANS', 'CPIAUCSL', 'Housing', 'NFCI', 'HML', 'RF']]
vif_FD_Stepwise = x_data[ ['Housing', 'NFCI', 'RF', 'CPIAUCSL' , 'HML']]

In [ ]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(
    vif_FD_forward.values, i) for i in range(vif_FD_forward.shape[1])]
vif["features"] = vif_FD_forward.columns
vif.round(2)

In [ ]:
## BUSLOANS 변수 제거후(vif_FD_backward) VIF 확인 

vif = pd.DataFrame()
vif_FD_forward= vif_FD_forward.drop('BUSLOANS',axis=1)
vif["VIF Factor"] = [variance_inflation_factor(
    vif_FD_forward.values, i) for i in range(vif_FD_forward.shape[1])]
vif["features"] = vif_FD_forward.columns
vif

In [ ]:
## M3 변수 제거후(vif_FD_backward) VIF 확인 

vif = pd.DataFrame()
vif_FD_forward= vif_FD_forward.drop('M3',axis=1)
vif["VIF Factor"] = [variance_inflation_factor(
    vif_FD_forward.values, i) for i in range(vif_FD_forward.shape[1])]
vif["features"] = vif_FD_forward.columns
vif
## 모두 10 이하의 VIF 값을 갖는다

In [ ]:
FD_model_forward = smf.ols(formula = 'F_D ~  Housing + NFCI + RF + CPIAUCSL + HML + SP500',data = FD_data)
FD_result_forward = FD_model_forward.fit()
FD_result_forward.summary()

In [ ]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(
    vif_FD_backward.values, i) for i in range(vif_FD_backward.shape[1])]
vif["features"] = vif_FD_backward.columns
vif.round(2)

In [ ]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(
    vif_FD_Stepwise.values, i) for i in range(vif_FD_Stepwise.shape[1])]
vif["features"] = vif_FD_Stepwise.columns
vif.round(2)

## Cross Validation

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
ED = y_data['E_D']
ED

In [ ]:
x_train_all, x_test, y_train_all, y_test = train_test_split(x_data, ED, test_size = 0.2, random_state = 42)

In [ ]:
x_train_all

In [ ]:
y_train_all

In [ ]:
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold

In [ ]:
n_splits = 5

cv = KFold(n_splits, shuffle=True, random_state=42)

In [ ]:
# Cross_validation 부분을 함수화
def cross_val(n_splits, cv, data, y , x):
    scores = np.zeros(n_splits)
    for i, (idx_train, idx_test) in enumerate(cv.split(data)):
        df_train = data.iloc[idx_train]
        df_test = data.iloc[idx_test]

        model = smf.ols(formula = f'{y} ~ {x}', data=df_train)
        result = model.fit()

        pred = result.predict(df_test)
        rsquared = r2_score(df_test[y], pred)

        scores[i] = rsquared

    return scores

### Euclidean

In [ ]:
ED = 'E_D'
ED_x = 'Real_Effective + SP500 + BUSLOANS + CPIAUCSL + CCI + Inflation + Housing + long_term + NFCI + M3 + Unemployment_Rate + SMB + HML + RMW + CMA + RF'
ED_scores = cross_val(n_splits,cv, data, ED, ED_x)
ED_scores

In [ ]:
print(ED_scores.mean())

### Cosine

In [ ]:
CD = 'C_D'
CD_x = 'Real_Effective + SP500 + BUSLOANS + CPIAUCSL + CCI + Inflation + Housing + long_term + NFCI + M3 + Unemployment_Rate + SMB + HML + RMW + CMA + RF'
CD_scores = cross_val(n_splits,cv, data, CD, CD_x)
CD_scores

In [ ]:
print(CD_scores.mean())

### Frechet

In [ ]:
FD = 'F_D'
FD_x = 'Real_Effective + SP500 + BUSLOANS + CPIAUCSL + CCI + Inflation + Housing + long_term + NFCI + M3 + Unemployment_Rate + SMB + HML + RMW + CMA + RF'
FD_scores = cross_val(n_splits,cv, data, FD, FD_x)
FD_scores

In [ ]:
print(FD_scores.mean())